# **OptCouple**

OptCouple is an algorithm that combines knock-outs, knock-ins and media additions to couple metabolic flux towards a target metabolite with biomass accumulation. Most of the code was extracted from reference [1].
## References
[1] Kristian Jensen, Valentijn Broeken, Anne Sofie Lærke Hansen, Nikolaus Sonnenschein, Markus J. Herrgård,
OptCouple: Joint simulation of gene knockouts, insertions and medium modifications for prediction of growth-coupled strain designs,
Metabolic Engineering Communications,
Volume 8,
2019,
e00087,
ISSN 2214-0301,
https://doi.org/10.1016/j.mec.2019.e00087.


In [5]:
# Load dependencies
from cobra.io import read_sbml_model
import os
os.environ["OPTLANG_USE_SYMENGINE"] = "False"
import cameo
import cobra
from cobra.io import load_model
from optlang.duality import convert_linear_problem_to_dual
import logging
from cameo.models import universal
from cameo.data import metanetx

In [6]:
import gurobipy

In [11]:
logger = logging.getLogger()

#cobra.show_versions()

In [73]:
# Read the E.coli model with NMN pathway
model = read_sbml_model('scripts/iML1515.xml')
model.reactions.ATPM.delete() # ATPM has forced flux bounds, which is not compatible with OptCouple

#Choosing uptake rates
medium=model.medium
medium['EX_ncam_e'] = 100
medium['EX_nr_e'] = 100
medium["EX_na_e"] = 100
medium["EX_glc__D_e"] = 10


In [74]:
model

Name,iML1515
Memory address,0x07f7963b8e040
Number of metabolites,1877
Number of reactions,2711
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [94]:
# # Select reactions to exclude from knock-out
# Make a set of reactions that you do not want to consider for knock-out, as they are spontaneous or diffusion reactions, or reactions that have no gene information. 
# Make sure that the target reaction is not in this set!
# Define reactions that should be excluded from knock-out as these will be impossible in practice.

def determine_reactions_to_exclude(model, spontaneous_gene_id=None, ids_not_to_exclude=None):
    """
    Creates a set of reaction ids from the cobra model that should not be suggested as modification by the algorithm.
    This set includes reactions that occur spontaneously, reactions that 
    have no gene information in the model 
    and transport reactions that occur as result of diffusion.
    
    Arguments:
    model: a cobra model
    spontaneous_gene_id: the gene id (string) that spontaneous reactions are assigned to
    ids_not_to_exclude: set of gene ids that should not be in the resulting set
    """
    
    exclude_reaction_ids = set()

    # Exclude reactions for which no gene is available, contain "diffusion" in their description
    # or are assigned to the spontaneous gene id.
    for r in model.reactions:
        if not r.genes or "diffusion" in r.name or spontaneous_gene_id in [g.id for g in r.genes]: 
            exclude_reaction_ids.add(r.id)
            
    # Make sure that the 'ids_not_to_exclude' reactions are not in the set      
    if ids_not_to_exclude is not None:
        for r_id in ids_not_to_exclude:
            exclude_reaction_ids.discard(r_id)

    return exclude_reaction_ids

In [ ]:
# run function excluded reactions
exclude_reaction_ids = determine_reactions_to_exclude(model, ids_not_to_exclude=['EX_mac_e', 'mac_etoEX', 'MSADH'])

In [77]:
# # Load universal model for knock-ins
# The universal model used is taken from cameo.models.metanetx_universal_model_bigg and converted into BiGG identifiers using the conversion dictionary cameo.data.metanetx.mnx2bigg, 
# as done in the "03 TheAlgorithm EcoliCore2 - methylation" notebook.

# Load a universal BiGG model from cameo, consisting of 2819 metabolites and 6416 reactions.
univ_model = cobra.io.load_json_model("/Users/shengbaowang/group-assignment-2021-group-8-malonic-acid-in-k-phaffii/src/models/gen/Universal_BiGG_model.json")
# univ_model

# Print all metabolites and reactions in a model. For debugging/searching:
# for m in univ_model.metabolites:
#     print(m.id, m.name, m.compartment, sep="\t")
# for r in univ_model.reactions:
#     print(r.id, r.name, r.reaction, r.compartments, sep="\t")

In [78]:
# # Add relevant universal reactions to the native model

def fuse_native_with_universal_model(native_model, universal_model, number_of_knockins, include_demands=False):
    """
    Merges the native cobra model with the relevant reactions from the universal_model. 
    The universal reactions that could be fully connected with the specified number of knockins are selected.
    The native reactions are flagged with a type attribute 'native'. Reactions from the universal model are assigned the 'heterologous' type.
    
    Arguments:
    native_model: the native cobra model
    universal_model: the universal cobra model to select knockins from
    include_demands: whether demand reactions from the universal model with matching reactants are added or not
    number_of_knockins: the number of knockins on which to base the selection of relevant universal reactions
    """
    # If no fusion of models is needed, but just labeling all native reactions with .type = "native":
    if number_of_knockins == 0:
        for r in native_model.reactions:
            r.type = "native"
        return native_model
    
    # Not to modify the models given as arguments
    fused_model = native_model.copy()
    reduced_universal_model = universal_model.copy()
    
    # Keep track of native and heterologous reactions by adding a .type attribute to each reaction:
    for r in fused_model.reactions:
        r.type = "native"
        
    # Make a set structure to identify reactions in the universal model that are duplicates of the native reactions
    # reactions = set(     # This set contains representations of all reactions
    #     frozenset(       # This frozenset contains 1 frozenset with all reactants and 1 with all products
    #         frozenset(   # These 2 frozensets contains all the metabolites in either products or reactants
    #             tuple(metabolite_id, coefficient)  # Coefficients are made positive if the reaction is reversible
    #         )                                      # Otherwise, reactant coefficients are negative
    #     )
    # )

    native_reactions_set = set()
    for r in fused_model.reactions:
        product_list = [] # Storing the metabolite ids for all products
        reactant_list = []
        prod_coef_list = [] # Storing the matching coefficients
        reac_coef_list = []
        # Make lists of the product and reactant ids and their coefficients
        for m in r.products:
            product_list.append(m.id)
        for m in r.reactants:
            reactant_list.append(m.id)        
        for c in r.get_coefficients(r.products):
            prod_coef_list.append(c)
        if r.reversibility:
            for c in r.get_coefficients(r.reactants):
                reac_coef_list.append(-c) # Remove negative sign from coefficients
        else:
            for c in r.get_coefficients(r.reactants):
                reac_coef_list.append(c) # Keep the negative sign to indicate direction
        # Zip these lists to make tuples of (metabolite, coefficient) pairs
        productset = frozenset(zip(product_list, prod_coef_list))
        reactantset = frozenset(zip(reactant_list, reac_coef_list))
        reactionset = set()
        reactionset.add(reactantset)
        reactionset.add(productset)
        native_reactions_set.add(frozenset(reactionset))
        #print(native_reactions_set)
    
    # Remove reactions from the universal model that are boundary reactions (if include_demands is not set to True), 
    # or duplicates of the native model used
    reactions_to_remove = set()
    for r in reduced_universal_model.reactions:
        # Create the same structure as above for this reaction. Compare if it is among the native reactions.
        product_list = []
        reactant_list = []
        prod_coef_list = []
        reac_coef_list = []
        # Make lists of the product and reactant ids and their coefficients
        for m in r.products:
            product_list.append(m.id)
        for m in r.reactants:
            reactant_list.append(m.id)        
        for c in r.get_coefficients(r.products):
            prod_coef_list.append(c)
        if r.reversibility:
            for c in r.get_coefficients(r.reactants):
                reac_coef_list.append(-c) # Remove negative sign from coefficients
        else:
            for c in r.get_coefficients(r.reactants):
                reac_coef_list.append(c) # Keep the negative sign to indicate direction
        # Zip these lists to make tuples of (metabolite, coefficient) pairs
        productset = frozenset(zip(product_list, prod_coef_list))
        reactantset = frozenset(zip(reactant_list, reac_coef_list))
        reactionset = set()
        reactionset.add(reactantset)
        reactionset.add(productset)
        
        if (r.products == [] and not include_demands) or                 reactionset in native_reactions_set:
            reactions_to_remove.add(r)

    reduced_universal_model.remove_reactions(reactions_to_remove)
    
    
    # Add the relevant reactions from universal model to the organism's native model, 
    # based on the connection of its metabolites to the native model:
    # For every 2 knock-ins allowed, add all reactions that have either all their reactants or all products present in the model so far.
    # This means that knock-in pathways are build from both start and end, which will connect at some point if enough knock-ins are allowed.
    # Filling the gap from two sides is needed to take reversible reactions into account, which cause problems if building only from the reactant side. 
    # The last knock-in (number_of_knockins = 1) allowed should be connected to the model by both all reactants and products in order to carry flux
    while number_of_knockins > 0:
        model_metabolite_ids = [m.id for m in fused_model.metabolites]
        for r in reduced_universal_model.reactions:
            if (number_of_knockins > 1 and all(m.id in model_metabolite_ids for m in r.reactants) or             (r.products != [] and all(m.id in model_metabolite_ids for m in r.products))) or             (number_of_knockins == 1 and all(m.id in model_metabolite_ids for m in r.metabolites)):    
                    add = r.copy()
                    add.type = "heterologous"
                    fused_model.add_reaction(add)
        number_of_knockins -= 2

               
    # Remove all reactions that can't carry flux in the end, 
    # to trim away all added heterologous reactions that are never relevant as a knock-in.
    blocked_reactions = cameo.flux_analysis.analysis.find_blocked_reactions(fused_model)
    fused_model.remove_reactions([r for r in blocked_reactions])
         
    return fused_model

In [79]:
fused_model = fuse_native_with_universal_model(model, univ_model, include_demands=False, number_of_knockins=0)
fused_model
# For some reason fusing these while allowing knock-ins removes reactions present in iMT1026v3 from the fused model

Name,iML1515
Memory address,0x07f7963b8e040
Number of metabolites,1877
Number of reactions,2711
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [80]:
 for r in fused_model.reactions:
     if r.type == "heterologous":
         print(r)
 for r in fused_model.reactions:
     if r.type != "native":
         print(r)

In [85]:
# # Specify metabolites for medium additions
medium_metabolites = []
# ["ala__L_c", "arg__L_c", "asn__L_c", "asp__L_c", "cys__L_c", "glu__L_c", "gln__L_c", "gly_c", "his__L_c", 
# "ile__L_c", "leu__L_c", "lys__L_c", "met__L_c", "phe__L_c", "pro__L_c", "ser__L_c", "thr__L_c", "trp__L_c", 
# "tyr__L_c", "val__L_c", "fru_c","mal__L_c", "lac__D_c", "ac_c"]
# 24 (L-)Amino acids, D-fructose, L-malate, D-lactate,  acetate

In [2]:
# # Create the MILP problem
# This is the main algorithm, creating a MILP problem that when solved returns one or more growth-coupled designs for the target reaction.
for reac in model.reactions:
    if reac.lower_bound > 0 or reac.upper_bound < 0:
        print(reac.id, "has a forced flux. This should be removed")
        
import optlang

def setup_growthcoupling_milp(model, target, medium_metabolites, K=2, L=1, M=1, exclude_reaction_ids=set(), remove_blocked=False):
    """
    This function constructs the MILP problem to predict growth-coupled designs with. 
    K is the number of knockouts that are allowed. L is the number of knockins allowed. M is the number of medium additions.
    exclude_reaction_ids takes a list of reaction ids that shouldn't be considered for knockout/in/medium addition 
    (i.e. spontaneous reactions and exchange reactions). These reactions are thus always allowed to have flux within their model bounds, so this should only be used for excluding knockouts.
    """
    # Check model for forced fluxes, which should not be present. The zero solution must be allowed.
    for r in model.reactions:
        if r.lower_bound > 0 or r.upper_bound < 0:
            raise ValueError("%s has a forced flux. Remove the reaction or change its bounds for the function to work." % r.id)
    
    # If a cobra.Reaction is given as target, make the target in just its ID string.
    if isinstance(target, cobra.core.Reaction):
        target = target.id
    
    original_model = model
    model = model.copy()
    
    # Add boundary reactions for these metabolites named "ADD_<metabolitename>" to the model, with bounds (-10,0).
    medium_addition_reactions = []   # List of the addition reactions added to the model
    for m in medium_metabolites:
        met = model.metabolites.get_by_id(m)
        r = model.add_boundary(met, type="medium addition", reaction_id=("ADD_"+m), lb=-10, ub=0)
        r.type = "medium"
        medium_addition_reactions.append(r)
    
    # Create a set of reactions that should not be considered for knockout (from exclude_from_knockouts list of ids)
    excluded_reactions = set()
    for r_id in exclude_reaction_ids:
        excluded_reactions.add(model.reactions.get_by_id(r_id))
    
    
    # Set the solver to Gurobi for the fastest result. Set to CPLEX if Gurobi is not available.
    if "gurobi" in cobra.util.solver.solvers.keys():
        logger.info("Changing solver to Gurobi and tweaking some parameters.")
        if "gurobi_interface" not in model.solver.interface.__name__:
            model.solver = "gurobi"
        # The tolerances are set to the minimum value. This gives maximum precision.
        problem = model.solver.problem
        problem.params.NodeMethod = 1 # primal simplex node relaxation
        problem.params.FeasibilityTol = 1e-9 #If a flux limited to 0 by a constraint, which range around it is still considered the same as 0 > set smallest possible
        problem.params.OptimalityTol = 1e-3 #how sure the solver has to be about this optimum being really the best it has.
        problem.params.IntFeasTol = 1e-9 #If a value is set to an integer, how much may it still vary? > set smallest possible
        problem.params.MIPgapAbs = 1e-9
        problem.params.MIPgap = 1e-9
        problem.params.TimeLimit = 200 # Use max 200 seconds when called, return best solution after that
        problem.params.PoolSearchMode = 1 #0 for only finding the optimum, 1 for finding more solutions (but no quality guaranteed), 2 for finding the n best possible solutions 
        #problem.params.PoolSolutions = 10 # Number of solutions kept when finding the optimal solution
        #problem.params.PoolGap = 0.9 # only store solutions within 90% of the optimal objective value

    elif "cplex" in cobra.util.solver.solvers.keys():
        logger.warning("Changing solver to CPLEX, as Gurobi is not available. This may cause a big slowdown and limit options afterwards.")
        if "cplex_interface" not in model.solver.interface.__name__:
            model.solver = "cplex"
        # The tolerances are set to the minimum value. This gives maximum precision.
        problem = model.solver.problem
        problem.parameters.mip.strategy.startalgorithm.set(1) # primal simplex node relaxation
        problem.parameters.simplex.tolerances.feasibility.set(1e-9) #If a flux limited to 0 by a constraint, which range around it is still considered the same as 0 > set smallest possible
        problem.parameters.simplex.tolerances.optimality.set(1e-3) #possibly fine with 1e-3, try if allowed. Is how sure the solver has to be about this optimum being really the best it has.
        problem.parameters.mip.tolerances.integrality.set(1e-9) #If a value is set to an integer, how much may it still vary? > set smallest possible
        problem.parameters.mip.tolerances.absmipgap.set(1e-9)
        problem.parameters.mip.tolerances.mipgap.set(1e-9)
        #problem.parameters.mip.pool.relgap.set(0.9) # For populate: find all solutions within 10% of the optimum for relgap = 0.1
        #problem.parameters.timelimit.set(200) # Use max 200 seconds for solving
        #problem.parameters.mip.limits.populate.set(20) # Find max 20 solutions (=default)
   
    else:
        logger.warning("You are trying to run 'OptCouple' with %s. This might not end well." %
                      model.solver.interface.__name__.split(".")[-1])
    
    # Remove reactions that are blocked: no flux through these reactions possible. This will reduce the search space for the solver, if not done already.
    if remove_blocked:
        blocked_reactions = cameo.flux_analysis.analysis.find_blocked_reactions(fused_model)
        fused_model.remove_reactions(blocked_reactions)
        logger.debug("Removed " + str(len(blocked_reactions)) + " reactions that were blocked")
    
    # Make dual
    model_without_target = model.copy() # This variable looks unnecessary, but is kept out of fear of messing stuff up
    model_without_target.optimize()
    dual_problem = convert_linear_problem_to_dual(model_without_target.solver)
    logger.debug("Dual problem successfully created")

    # Combine primal and dual
    primal_problem = model.solver

    for var in dual_problem.variables:  # All variables in the dual are copied to the primal
        var = primal_problem.interface.Variable.clone(var)
        primal_problem.add(var)
    for const in dual_problem.constraints:  # All constraints in the dual are copied to the primal
        const = primal_problem.interface.Constraint.clone(const, model=primal_problem)
        primal_problem.add(const)
    logger.debug("Dual and primal combined")

    dual_problem.optimize()
    
    
    # Dictionaries to hold the binary control variables: 
    native_y_vars = {}       # 1 for knockout, 0 for active
    heterologous_y_vars = {} # 1 for knockin, 0 for inactive
    medium_y_vars = {}       # 1 for medium addition (up to -10), 0 for no addition
    
    # Now the fun stuff
    constrained_dual_vars = set()
    
    # Fill the dictionaries with binary variables
    # For the knockouts: only for the native reactions which are not exchanges
    for reaction in [r for r in model.reactions - excluded_reactions if r.type == "native"]:

        # Add constraint variables
        interface = model.solver.interface
        y_var = interface.Variable("y_" + reaction.id, type="binary")

        # Constrain the primal: flux through reactions maximum within (-1000, 1000), or smaller boundaries defined before
        model.solver.add(interface.Constraint(reaction.flux_expression - 1000 * (1 - y_var), ub=0, name="primal_y_const_"+reaction.id+"_ub"))
        model.solver.add(interface.Constraint(reaction.flux_expression + 1000 * (1 - y_var), lb=0, name="primal_y_const_"+reaction.id+"_lb")) 

        # Constrain the dual
        constrained_vars = []

        if reaction.upper_bound != 0:
            dual_forward_ub = model.solver.variables["dual_" + reaction.forward_variable.name + "_ub"]
            model.solver.add(interface.Constraint(dual_forward_ub - 1000 * y_var, ub=0))
            constrained_vars.append(dual_forward_ub)
        if reaction.lower_bound != 0:
            dual_reverse_ub = model.solver.variables["dual_" + reaction.reverse_variable.name + "_ub"] 
            model.solver.add(interface.Constraint(dual_reverse_ub - 1000 * y_var, ub=0))
            constrained_vars.append(dual_reverse_ub)
        constrained_dual_vars.update(constrained_vars)

        # Add to dictionary with binary variables for native reactions:
        native_y_vars[y_var] = reaction


    # For the knockins and medium additions:
    for reaction in [r for r in model.reactions - excluded_reactions if r.type == "heterologous" or r.type == "medium"]:          
        # Add constraint variables
        interface = model.solver.interface
        y_var = interface.Variable("y_" + reaction.id, type="binary")

        # Constrain the primal: flux through reactions maximum within (-1000, 1000), or smaller boundaries defined before
        model.solver.add(interface.Constraint(reaction.flux_expression - 1000 * y_var, ub=0, name="primal_y_const_"+reaction.id+"_ub"))
        model.solver.add(interface.Constraint(reaction.flux_expression + 1000 * y_var, lb=0, name="primal_y_const_"+reaction.id+"_lb"))  

        # Constrain the dual
        constrained_vars = []

        if reaction.upper_bound != 0:
            dual_forward_ub = model.solver.variables["dual_" + reaction.forward_variable.name + "_ub"]
            model.solver.add(interface.Constraint(dual_forward_ub - 1000 * (1 - y_var), ub=0))
            constrained_vars.append(dual_forward_ub)
        if reaction.lower_bound != 0:
            dual_reverse_ub = model.solver.variables["dual_" + reaction.reverse_variable.name + "_ub"]
            model.solver.add(interface.Constraint(dual_reverse_ub - 1000 * (1 - y_var), ub=0))
            constrained_vars.append(dual_reverse_ub)
        constrained_dual_vars.update(constrained_vars)

        # Add y variable to the corresponding modifications dictionary
        if reaction.type == "medium":
            medium_y_vars[y_var] = reaction         
        elif reaction.type == "heterologous":
            heterologous_y_vars[y_var] = reaction
            
    logger.debug("Control variables created")
    

    # Set the objective
    primal_objective = model.solver.objective
    dual_objective = interface.Objective.clone(
        dual_problem.objective, model=model.solver
    )

    reduced_expression = optlang.symbolics.Add(
        *((c * v) for v, c in dual_objective.expression.as_coefficients_dict().items()
        if v not in constrained_dual_vars)
    )
    dual_objective = interface.Objective(reduced_expression, direction=dual_objective.direction)

    full_objective = interface.Objective(primal_objective.expression - dual_objective.expression, direction="max")
    model.objective = full_objective
    logger.debug("Objective created")
    
        
    # Add number of knockouts constraint. ub=K+1 as the target will be forced to be 1 as well
    knockout_number_constraint = model.solver.interface.Constraint(
        optlang.symbolics.Add(*native_y_vars), lb=0, ub=K+1, name="number_of_knockouts_constraint"
    )
    model.solver.add(knockout_number_constraint)
   
    # Add number of knockins constraint
    knockin_number_constraint = model.solver.interface.Constraint(
        optlang.symbolics.Add(*heterologous_y_vars), lb=0, ub=L, name="number_of_knockins_constraint"
    )
    model.solver.add(knockin_number_constraint)
  
    # Add number of medium additions constraint
    medium_additions_number_constraint = model.solver.interface.Constraint(
        optlang.symbolics.Add(*medium_y_vars), lb=0, ub=M, name="number_of_medium_additions_constraint"
    )
    model.solver.add(medium_additions_number_constraint)
    
    logger.debug("Added constraint for number of knockouts, knockins and medium additions")

    
    # Force target control variable to be 1, but allow flux in primal
    model.solver.constraints["primal_y_const_" + target + "_lb"].lb = -2000
    model.solver.constraints["primal_y_const_" + target + "_ub"].ub = 2000
    model.variables["y_" + target].lb = 1
    
    return model, native_y_vars, heterologous_y_vars, medium_y_vars

NameError: name 'model' is not defined

In [87]:
fused_model

Name,iML1515
Memory address,0x07f7963b8e040
Number of metabolites,1877
Number of reactions,2711
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [1]:
# Run the function
milp, native_y_vars, heterologous_y_vars, medium_y_vars = setup_growthcoupling_milp(fused_model, 'NAMPT', medium_metabolites, K=2, L=0, M=0, remove_blocked=False, exclude_reaction_ids=exclude_reaction_ids)

NameError: name 'setup_growthcoupling_milp' is not defined